In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("Cleaned_Table.csv")

In [4]:
df

,gmv,units,payment_type,product_analytic_sub_category,product_mrp,Holiday_Flag,Pay_Date_Flag,climate,is_colder,weather_affect,is_delayed
0,3149.0,1,COD,HomeAudio,6990,0,0,2,0,0,0
1,350.0,1,COD,CameraAccessory,399,0,0,3,0,1,0
2,2800.0,1,COD,HomeAudio,3000,0,0,3,0,1,0
3,385.0,1,Prepaid,GamingAccessory,699,1,1,3,0,0,0
4,428.0,1,Prepaid,GamingAccessory,999,1,1,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
540602,1399.0,1,COD,HomeAudio,1999,0,0,3,0,1,1
540603,699.0,1,COD,GamingAccessory,999,0,0,3,0,0,0
540604,215.0,1,COD,CameraAccessory,1999,0,0,3,0,0,0
540605,169.0,1,COD,CameraAccessory,499,0,1,3,0,0,1


In [5]:
df["payment_type"] = df["payment_type"].apply(lambda i : 1 if i == "COD" else 0)

In [6]:
df_CA = df[df["product_analytic_sub_category"] == "CameraAccessory"]

In [7]:
df_GA = df[df["product_analytic_sub_category"] == "GamingAccessory"]

In [8]:
df_HA = df[df["product_analytic_sub_category"] == "HomeAudio"]

In [9]:
df_GA['gmv'] =  np.log1p(df_GA['gmv'])

<ipython-input-9-4457b5d61c49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GA['gmv'] =  np.log1p(df_GA['gmv'])


In [10]:
df_CA['gmv'] =  np.log1p(df_CA['gmv'])

<ipython-input-10-32df8c37ef4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_CA['gmv'] =  np.log1p(df_CA['gmv'])


In [11]:
df_HA['gmv'] =  np.log1p(df_HA['gmv'])

<ipython-input-11-8ae4139188d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_HA['gmv'] =  np.log1p(df_HA['gmv'])


In [12]:
df_CA['gmv'].skew()

-0.06411280695810176

In [13]:
df_GA['gmv'].skew()

0.047081548797294606

In [14]:
df_HA['gmv'].skew()

-0.8289998180346324

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [27]:
def algorithm_train_test_accuracy(x_train, x_test, y_train, y_test, algorithm):

    model = algorithm().fit(x_train, y_train)
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)

    # algo = str(algorithm).split("'")[1].split(".")[-1]
    accuracy = {'algorithm': algorithm.__name__,
                'R2_train' : r2_train,
                'R2_test'  : r2_test}

    return accuracy

In [16]:
def ml_regression(df):
    x = df.drop(columns=['gmv','product_analytic_sub_category'], axis=1)
    y = df['gmv']
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
    print(algorithm_train_test_accuracy(x_train, x_test, y_train, y_test, DecisionTreeRegressor))
    print(algorithm_train_test_accuracy(x_train, x_test, y_train, y_test, ExtraTreesRegressor))
    print(algorithm_train_test_accuracy(x_train, x_test, y_train, y_test, RandomForestRegressor))
    print(algorithm_train_test_accuracy(x_train, x_test, y_train, y_test, AdaBoostRegressor))
    print(algorithm_train_test_accuracy(x_train, x_test, y_train, y_test, GradientBoostingRegressor))
    print(algorithm_train_test_accuracy(x_train, x_test, y_train, y_test, XGBRegressor))

In [28]:
# Camera Accesory
ml_regression(df_CA)

{'algorithm': 'DecisionTreeRegressor', 'R2_train': 0.7445218066160006, 'R2_test': 0.7183756417742319}
{'algorithm': 'ExtraTreesRegressor', 'R2_train': 0.7445218066160006, 'R2_test': 0.7165149987794325}
{'algorithm': 'RandomForestRegressor', 'R2_train': 0.7436040911092112, 'R2_test': 0.7211445556577982}
{'algorithm': 'AdaBoostRegressor', 'R2_train': 0.39619758911493663, 'R2_test': 0.3963323588830535}
{'algorithm': 'GradientBoostingRegressor', 'R2_train': 0.6307521088538877, 'R2_test': 0.6281955711282325}
{'algorithm': 'XGBRegressor', 'R2_train': 0.6936722700539186, 'R2_test': 0.6869358861979971}


In [29]:
# Gaming Accesory
ml_regression(df_GA)

{'algorithm': 'DecisionTreeRegressor', 'R2_train': 0.8292178463759462, 'R2_test': 0.8144440862319668}
{'algorithm': 'ExtraTreesRegressor', 'R2_train': 0.8292178463759462, 'R2_test': 0.8138275570050946}
{'algorithm': 'RandomForestRegressor', 'R2_train': 0.8286222089823558, 'R2_test': 0.8170815015035904}
{'algorithm': 'AdaBoostRegressor', 'R2_train': 0.44464750632921846, 'R2_test': 0.4468519528235325}
{'algorithm': 'GradientBoostingRegressor', 'R2_train': 0.7525911959833282, 'R2_test': 0.7531962115484925}
{'algorithm': 'XGBRegressor', 'R2_train': 0.8062331129997128, 'R2_test': 0.8028998659334794}


In [52]:
# Home Audio
ml_regression(df_HA)

{'algorithm': 'DecisionTreeRegressor', 'R2_train': 0.9524035539237486, 'R2_test': 0.9405552876045974}
{'algorithm': 'ExtraTreesRegressor', 'R2_train': 0.9524035539237486, 'R2_test': 0.9399100315990049}
{'algorithm': 'RandomForestRegressor', 'R2_train': 0.9519539671255625, 'R2_test': 0.941972887356858}
{'algorithm': 'AdaBoostRegressor', 'R2_train': 0.8227991871498812, 'R2_test': 0.8238971365496934}
{'algorithm': 'GradientBoostingRegressor', 'R2_train': 0.9064630776290175, 'R2_test': 0.9074255696794873}
{'algorithm': 'XGBRegressor', 'R2_train': 0.9351517943196126, 'R2_test': 0.9295737816877844}


In [ ]:
# From these observations Random Forest Regressor give best model. Adjusting the hyper parameter will give better result

In [54]:
def random_forest(df):
    x = df.drop(columns=['gmv','product_analytic_sub_category'], axis=1)
    y = df['gmv']
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x_scaled,y,test_size=0.2,random_state=42)

    param_grid = {
        'max_depth': [30,50],
        'min_samples_split': [2],
        'min_samples_leaf': [2]
    }
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='r2')
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test)
    print("Best hyperparameters:", grid_search.best_params_)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print('Mean squared error:', mse)
    print('Root Mean squared error:', np.sqrt(mse))
    print('R-squared:', r2)
    return best_model

In [47]:
model_CA = random_forest(df_CA)

Best hyperparameters: {'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mean Absolute Error: 0.24843042092895382
Mean squared error: 0.17255763123486648
Root Mean squared error: 0.4154005672057592
R-squared: 0.7222769928394428


In [48]:
model_GA = random_forest(df_GA)

Best hyperparameters: {'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mean Absolute Error: 0.22256618695127672
Mean squared error: 0.1385749314731455
Root Mean squared error: 0.37225653986618623
R-squared: 0.8183803252306376


In [55]:
model_HA = random_forest(df_HA)

Best hyperparameters: {'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mean Absolute Error: 0.09141548763019472
Mean squared error: 0.024406893217550123
Root Mean squared error: 0.15622705661168337
R-squared: 0.9427472953824467
          feature  importance
2     product_mrp    0.973440
5         climate    0.007226
0           units    0.006534
8      is_delayed    0.004897
6       is_colder    0.003088
1    payment_type    0.002054
7  weather_affect    0.001516
4   Pay_Date_Flag    0.000825
3    Holiday_Flag    0.000420


In [ ]:
# Deep Learning

In [17]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import LeakyReLU, Dropout
from tensorflow.keras import backend as K
import keras_tuner as kt

In [19]:
def deep_learning(df):
    x = df.drop(columns=['gmv','product_analytic_sub_category'], axis=1)
    y = df['gmv']
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    x_train, x_test, y_train, y_test = train_test_split(x_scaled,y,test_size=0.2,random_state=42)

    # Defining the ANN model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(x_train.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='linear')
    ])

    # Compile the model with learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='mean_squared_error')

    # Train the model with EarlyStopping and Reducing the Learning Rate
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=30, batch_size=64, callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
    ])


    y_pred = model.predict(x_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    metrics = {
        'R2': r2,
        'Mean Absolute Error': mae,
        'Mean Squared Error': mse,
        'Root Mean Squared Error': rmse
    }

    print(metrics)


In [71]:
deep_learning(df_HA)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1468/1468 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 5.5756 - val_loss: 0.3647 - learning_rate: 5.0000e-04
Epoch 2/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.8283 - val_loss: 0.3332 - learning_rate: 5.0000e-04
Epoch 3/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 0.6602 - val_loss: 0.2963 - learning_rate: 5.0000e-04
Epoch 4/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.5048 - val_loss: 0.2825 - learning_rate: 5.0000e-04
Epoch 5/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.3662 - val_loss: 0.2162 - learning_rate: 5.0000e-04
Epoch 6/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.2559 - val_loss: 0.1635 - learning_rate: 5.0000e-04
Epoch 7/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.1852 - val_loss: 0.1263 - learning_rate: 5.0000e-04
Epoch 8/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 0.1520 - val_loss: 0.1156 - learning_rate: 5.0000e-04
Epoch 9/30
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.1340 - val_l

In [20]:
deep_learning(df_CA)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2811/2811 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 4.4062 - val_loss: 0.6123 - learning_rate: 5.0000e-04
Epoch 2/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 0.8271 - val_loss: 0.4975 - learning_rate: 5.0000e-04
Epoch 3/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.5960 - val_loss: 0.4444 - learning_rate: 5.0000e-04
Epoch 4/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - loss: 0.4603 - val_loss: 0.3695 - learning_rate: 5.0000e-04
Epoch 5/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.4003 - val_loss: 0.3529 - learning_rate: 5.0000e-04
Epoch 6/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.3749 - val_loss: 0.3386 - learning_rate: 5.0000e-04
Epoch 7/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.3582 - val_loss: 0.3379 - learning_rate: 5.0000e-04
Epoch 8/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - loss: 0.3531 - val_loss: 0.3248 - learning_rate: 5.0000e-04
Epoch 9/30
2811/2811 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 0.3433 - val

In [21]:
deep_learning(df_GA)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2480/2480 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 6.5742 - val_loss: 0.5405 - learning_rate: 5.0000e-04
Epoch 2/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.8663 - val_loss: 0.4536 - learning_rate: 5.0000e-04
Epoch 3/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.6439 - val_loss: 0.4082 - learning_rate: 5.0000e-04
Epoch 4/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.4864 - val_loss: 0.3415 - learning_rate: 5.0000e-04
Epoch 5/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.3988 - val_loss: 0.3138 - learning_rate: 5.0000e-04
Epoch 6/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.3486 - val_loss: 0.2934 - learning_rate: 5.0000e-04
Epoch 7/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.3328 - val_loss: 0.2887 - learning_rate: 5.0000e-04
Epoch 8/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 0.3211 - val_loss: 0.2893 - learning_rate: 5.0000e-04
Epoch 9/30
2480/2480 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.3093 - val

For each category, Based on the R-squared values, the RandomForestRegressor emerged as the best model. The DecisionTreeRegressor and ExtraTreesRegressor also performed well. Models like XGBRegressor and GradientBoostingRegressor showed moderate performance, while AdaBoostRegressor underperformed . The deep learning model gives competitive performance but not surpassing RandomForest.